<a href="https://colab.research.google.com/github/sergeymasl/my_lifehacks/blob/main/etl_off_work.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#! pip install pyTelegramBotAPI
import pandas as pd
import numpy as np
import datetime
#import smtplib
#from email.mime.text import MIMEText
#from email.mime.multipart import MIMEMultipart
import telebot
import dataframe_image as dfi
import os

In [ ]:
def send_to_off_time_yes(df, description):
    '''
    Функция отправляет изображение датафрейма и описание в канал переработок
    '''
    # создание изображения датафрейма
    g = dfi.export(df, 'dataframe.png')
    # инициализация телебота
    bot = telebot.TeleBot('5271411201:AAEFlg6hKq2tLKG1HjQe-PthAMKOqayfBxk')
    # имя канала
    channel_name = '-1001662096375'
    # фото для отправки в бинарном виде
    doc = open('dataframe.png', 'rb')
    # отправка фото в канал
    bot.send_photo(channel_name, doc, caption=description, parse_mode="html")
    # закрываем и удаляем файл
    doc.close()
    os.remove('dataframe.png')

In [ ]:
def send_to_off_time_no(m_y):
    '''
    Функция отправляет сообщеине о  том что переработок нет
    '''
    # инициализация телебота
    bot = telebot.TeleBot('5271411201:AAEFlg6hKq2tLKG1HjQe-PthAMKOqayfBxk')
    # имя канала
    channel_name = '-1001662096375'
    # отправка сообщения в канал
    bot.send_message(channel_name, text = f"В этом месяце <b>({m_y})</b> нет переработок", parse_mode="html")

In [ ]:
#def send_message(send):
#    #создание объекта mime
#    msg = MIMEMultipart()
#    # создание самого сообщения
#    part = MIMEText(send, 'html')
#    # прикрепление сообщения к объекту mime
#    msg.attach(part)
#    # подключение к серверу электронной почты .mail 
#    smtpObj = smtplib.SMTP('smtp.mail.ru', 587)
#    # шифрование
#    smtpObj.starttls()
#    # авторизация
#    smtpObj.login('serezha_maslov@mail.ru','Sn5KnL5bMds7FhhNeRv9')
#    # отправка сообщения
#    smtpObj.sendmail('serezha_maslov@mail.ru', 'serezha_maslov@mail.ru', msg.as_string())
#    smtpObj.quit()

In [ ]:
# чтение файла
df_work = pd.read_excel(r'C:\Users\nt_user\Desktop\Переработки.xlsx')
# создание столбца с месяцем и годом
df_work['m-y'] = df_work['Дата'].apply(lambda x: str(x.month) + '-' + str(x.year))

In [ ]:
# создание переменных с текщим месяцм и текущим годом
m_y_now = str(datetime.date.today().month) + '-' + str(datetime.date.today().year)

In [ ]:
# проверка на наличие переработок
swith_off_time = False
if m_y_now in df_work['m-y'].to_list():
    swith_off_time = True

# создание таблицы переработок за текущий месяц
if swith_off_time:
    df_off_time_cur = df_work[df_work['m-y'] == m_y_now]
    df_off_time_cur.fillna('-',inplace=True)
    df_for_send = df_off_time_cur.copy()

# создание суммарного количества часов
if swith_off_time:
    timedelta_time_off = datetime.timedelta()
    for i in df_off_time_cur.index:
        hours_time_off = df_off_time_cur['Время переработки'][i].hour
        minutes_time_off = df_off_time_cur['Время переработки'][i].minute
        timedelta_time_off += datetime.timedelta(hours = hours_time_off, minutes = minutes_time_off)  

    minutes_delta_time_off = timedelta_time_off.seconds / 60
    hours_sum_time_off = int(minutes_delta_time_off // 60)
    minutes_sum_time_off = int(minutes_delta_time_off % 60)
    time_for_message = f'Общая сумма переработок в этом месяце (<b>{m_y_now}</b>): \n<b>{hours_sum_time_off} часа {minutes_sum_time_off} минут</b>'


# подготовка сообщения
#if swith_off_time:
#    send = f"""\
#        <html>
#          <head></head>
#          <body>
#          <p>Пора проверить переработки</p>
#          <p>{time_for_message}</p>
#            {df_for_send.to_html()}
#          </body>
#        </html>
#        """
#else:
#    send = 'Проверять нехера. Ты в этом месяце не перерабатывал.'


# отправка сообщения
if swith_off_time:
    send_to_off_time_yes(df_off_time_cur, time_for_message)
else:
    send_to_off_time_no(m_y_now)

In [ ]:
#send_message(send)